# TREC CrisisFACTs Track 2022 Tutorial

This notebook illustrates how to download the TREC 2022 CrisisFACTs event streams along with the information needs for each one.

## Downloading the Track Data

Below, we walk you through the steps for downloading the CrisisFACTS data using the `ir_datasets` package.

After downloading, we demonstrate converting this data into a Pandas DataFrame for quick inspect of the content associated with a given event-day pair.

<hr> 


**Part 1: Installing Needed Packages**

Before we can get the data, we need to install some packages to handle the download process. In particular, we are going to install one main package:

*   ir_datasets (https://github.com/allenai/ir_datasets): A python package that provides a common interface to many IR ad-hoc ranking benchmarks, training datasets, etc. We can use this to download the raw event streams and information needs for each.


In [1]:
!pip install git+https://github.com/allenai/ir_datasets.git@crisisfacts # install ir_datasets (crisisfacts branch)


<hr> 



**Part 2: Initalizing Your Credentials**

When you want to download part of the CrisisFACTs dataset we require that you provide a set of contact details. The reason for this is two-fold: 1) the terms of service from some of the platforms (like Twitter) from which we have sourced data require us to do so, and 2) it allows us to collect statistics on how many people are making use of the data we provide.

**GDPR Statement**: By downloading the CrisisFACTs datasets, you agree to the University of Glasgow processing your personal data, as defined by the EU General Data Protection Regulation (GDPR) - your name and email in this case. Queries about data processing and access/deletion requests should be sent to [me via email](http://www.dcs.gla.ac.uk/~richardm/Home/Contact.html). We will store your data for as long as the track is on-going and up-to 2 years beyond that. I may contact you using the details provided to notify you about changes in the datasets or track, to provide information or ask you questions about your participation or otherwise contact you about topics relevant to emergency management. We may collate statistics from the provided information that will be published, but we will not release individual names or email addresses. 

Rather than entering these details every time you request the dataset, its more effcient to set this once up-front, so fill in your details below:

In [2]:
credentials = {
    "institution": "University of Foo", # University, Company or Public Agency Name
    "contactname": "Foo Bar", # Your Name
    "email": "foo@bar.edu", # A contact email address
    "institutiontype": "Research" # Either 'Research', 'Industry', or 'Public Sector'
}

# Write this to a file so it can be read when needed
import json
import os

home_dir = os.path.expanduser('~')

!mkdir -p ~/.ir_datasets/auth/
with open(home_dir + '/.ir_datasets/auth/crisisfacts.json', 'w') as f:
    json.dump(credentials, f)

<hr> 

**Part 3: Understanding the structure of the CrisisFACTs Dataset**

The CrisisFACTs dataset is divided into events, representing real-world crises. Each event is given an identifier, e.g. 'CrisisFACTS-001' is the Lilac Wildfire from 2017. We sometimes refer to the event number or 'eventNo', this is the last three digits of the event identifier, e.g. '001'. There are 8 events for CrisisFACTs 2022:

In [3]:
# Event numbers as a list
eventNoList = [
          "001", # Lilac Wildfire 2017
          "002", # Cranston Wildfire 2018
          "003", # Holy Wildfire 2018
          "004", # Hurricane Florence 2018
          "005", # 2018 Maryland Flood
          "006", # Saddleridge Wildfire 2019
          "007", # Hurricane Laura 2020
          "008" # Hurricane Sally 2020
]

Each event has a duration, i.e. it lasts for a number of days. In the CrisisFACTs track, you need to produce a timeline summary for each day for a set of events. You can get the list of days for an event as shown below (example is for event "001", i.e. the Lilac Wildfire 2017):

In [4]:
import requests

# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):

  # We will download a file containing the day list for an event
  url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

  # Download the list and parse as JSON
  dayList = requests.get(url).json()

  # Print each day
  # Note each day object contains the following fields
  #   {
  #      "eventID" : "CrisisFACTS-001",
  #      "requestID" : "CrisisFACTS-001-r3",
  #      "dateString" : "2017-12-07",
  #      "startUnixTimestamp" : 1512604800,
  #      "endUnixTimestamp" : 1512691199
  #   }

  return dayList

for day in getDaysForEventNo(eventNoList[0]):
  print(day["dateString"])

2017-12-07
2017-12-08
2017-12-09
2017-12-10
2017-12-11
2017-12-12
2017-12-13
2017-12-14
2017-12-15


For each day, we collected related content to the event from the following sources:


*   **Twitter**: We are re-using tweets collected as part of the TREC Incident Streams track (http://trecis.org). These tweets were crawled by keyword, and as such most are likely to be relevant to the event, but are not nessessaraly good candidates for inclusion into a summary of what is happening.
*   **Reddit**: Discussions regarding what happens during events also occurs on the forum platform Reddit. We collected relevant Reddit threads to each event, where we include both the original submission and subsequent comments within those threads.
*   **News**: Traditional news agencies are often a good source of information during an emergency and so we have also included a small number of news articles collected during each event as well.
*   **Facebook**: We collected Facebook/Meta posts from public pages that are relevant to each event using CrowdTangle. We cannot share the content of these posts, however, we have included the post and page ids of this content within the stream for those who have access to the CrowdTangle API and can retrieve this data separately. 

Because these sources have different formatting and characteristics, we reformatted this data into a list of standardized 'stream items', where a stream item contains:


*   **event**: The identifier of the event, e.g. 'CrisisFACTS-001'
*   **streamID**: A unique identifier for the stream item. This will generally be of the form 'CrisisFACTS-\<eventNo\>-\<source\>-\<postID\>-\<sentenceID\>', e.g. CrisisFACTS-001-Twitter-15712-0.
*   **unixTimestamp**: This is the time that the content was originally posted, expressed as a unix timestamp in seconds (UTC timezone).
*   **text**: The text of the stream item. The maximum length of a stream item is 200 characters. 
*   **sourceType**: A string denoting the source, i.e. either Twitter, Reddit, News or Facebook.
*   **source**: This is the original post content formated as JSON (ir_datasets ignores this field).

Since, some types of content are longer than others (compare a news article vs. a tweet for instance), for long-form content we perform sentence segmentation, so one input post might form multiple stream items. In these cases, the 'sentenceID' component of the streamID denotes the number of the sentence in the source content.


The dataset is structured by day and event. To access the stream items for a particular \<event,day\> pair we generate a request string specifying the day and event we want, of the form:

*   '**crisisfacts/\<eventNo\>/\<day\>**'

For instance, we could generate request strings for all CrisisFACTs \<event,day\> pairs as follows:

In [5]:
for eventNo in eventNoList: # for each event
  dayList = getDaysForEventNo(eventNo) # get the list of days
  print("Event "+eventNo)
  for day in dayList: # for each day
    print("  crisisfacts/"+eventNo+"/"+day["dateString"]) # construct the request string
  print()

Event 001
  crisisfacts/001/2017-12-07
  crisisfacts/001/2017-12-08
  crisisfacts/001/2017-12-09
  crisisfacts/001/2017-12-10
  crisisfacts/001/2017-12-11
  crisisfacts/001/2017-12-12
  crisisfacts/001/2017-12-13
  crisisfacts/001/2017-12-14
  crisisfacts/001/2017-12-15

Event 002
  crisisfacts/002/2018-07-25
  crisisfacts/002/2018-07-26
  crisisfacts/002/2018-07-27
  crisisfacts/002/2018-07-28
  crisisfacts/002/2018-07-29
  crisisfacts/002/2018-07-30

Event 003
  crisisfacts/003/2018-08-06
  crisisfacts/003/2018-08-07
  crisisfacts/003/2018-08-08
  crisisfacts/003/2018-08-09
  crisisfacts/003/2018-08-10
  crisisfacts/003/2018-08-12
  crisisfacts/003/2018-08-13

Event 004
  crisisfacts/004/2018-09-01
  crisisfacts/004/2018-09-04
  crisisfacts/004/2018-09-05
  crisisfacts/004/2018-09-07
  crisisfacts/004/2018-09-08
  crisisfacts/004/2018-09-09
  crisisfacts/004/2018-09-10
  crisisfacts/004/2018-09-11
  crisisfacts/004/2018-09-12
  crisisfacts/004/2018-09-13
  crisisfacts/004/2018-09-14


Now that we know what the request strings for each event and day are, we can download for the associated stream for each via ir_datasets:

In [6]:
import ir_datasets

# download the first day for event 001 (this is a lazy call, it won't download until we first request a document from the stream)
dataset = ir_datasets.load('crisisfacts/001/2017-12-07')

for item in dataset.docs_iter()[:10]: # create an iterator over the stream containing the first 10 items
  print(item)

CrisisFactsStreamDoc(doc_id='CrisisFACTS-001-News-5-0', event='CrisisFACTS-001', text='Live updates: San Diego County fire is 92 percent contained', source_type='News', unix_timestamp=1512604800)
CrisisFactsStreamDoc(doc_id='CrisisFACTS-001-News-5-1', event='CrisisFACTS-001', text='The Lilac fire now 92 percent contained, Cal Fire officials said Tuesday morning.', source_type='News', unix_timestamp=1512604800)
CrisisFactsStreamDoc(doc_id='CrisisFACTS-001-News-5-2', event='CrisisFACTS-001', text='The county of San Diego has opened a Local Assistance Center to help victims of the fire begin the rebuilding and recovery process.', source_type='News', unix_timestamp=1512604800)
CrisisFactsStreamDoc(doc_id='CrisisFACTS-001-News-5-3', event='CrisisFACTS-001', text='The center is at the Vista branch library on 700 Eucalyptus Avenue and will be open from 9:00 a.m. to 6:00 p.m.  Services offered will include crisis counseling and referral services, short term housi', source_type='News', unix_tim

As we can see the first stream items are tweets, and not all of them are relevant, particularly at the begining of the event. If we wanted to find content of other types we can try filtering by the source_type field.

In [7]:
import pandas as pd

# Convert the stream of items to a Pandas Dataframe
itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())

# Create a filter expression
is_reddit =  itemsAsDataFrame['source_type']=="Reddit"

# Apply our filter
itemsAsDataFrame[is_reddit]

,doc_id,event,text,source_type,unix_timestamp
3094,CrisisFACTS-001-Reddit-s13-0,CrisisFACTS-001,Lilac Fire requiring evacuations in Bonsall an...,Reddit,1512677642
4619,CrisisFACTS-001-Reddit-s14-0,CrisisFACTS-001,The Latest: Fire in San Diego County triggers ...,Reddit,1512683949
4635,CrisisFACTS-001-Reddit-s38-c1-0,CrisisFACTS-001,"Is it just me, or does it seem like California...",Reddit,1512683968
4636,CrisisFACTS-001-Reddit-s38-c2-0,CrisisFACTS-001,This is what causes the great San Diego migrat...,Reddit,1512683968
4637,CrisisFACTS-001-Reddit-s38-c3-0,CrisisFACTS-001,"North County near Bonsall, outside Oceanside, ...",Reddit,1512683968
7109,CrisisFACTS-001-Reddit-s7-0,CrisisFACTS-001,North county San Diego fires are just starting...,Reddit,1512690661


In [8]:
# Create a filter expression
is_twitter =  itemsAsDataFrame['source_type']=="Twitter"

# Apply our filter
itemsAsDataFrame[is_twitter]

,doc_id,event,text,source_type,unix_timestamp
351,CrisisFACTS-001-Twitter-15712-0,CrisisFACTS-001,The homie tell me meet him at a time and this ...,Twitter,1512604832
352,CrisisFACTS-001-Twitter-31905-0,CrisisFACTS-001,A couple tattoos from the other day. Thank yo...,Twitter,1512604855
353,CrisisFACTS-001-Twitter-14023-0,CrisisFACTS-001,Big increase in the wind plus drop in humidity...,Twitter,1512604876
354,CrisisFACTS-001-Twitter-31850-0,CrisisFACTS-001,Blue is the color of the shirt of the man i lo...,Twitter,1512604920
355,CrisisFACTS-001-Twitter-27052-0,CrisisFACTS-001,Prayers go out to you all! From surviving 2 ma...,Twitter,1512604977
...,...,...,...,...,...
7283,CrisisFACTS-001-Twitter-14176-0,CrisisFACTS-001,HEY SAN DIEGO WHAT ABOUT WE ALL TURN ON OUR SP...,Twitter,1512691188
7284,CrisisFACTS-001-Twitter-13471-0,CrisisFACTS-001,"Oh good, fires down by my parents in San Diego...",Twitter,1512691189
7285,CrisisFACTS-001-Twitter-34602-0,CrisisFACTS-001,"#LilacFire grows to 2,000 acres, is 0% contain...",Twitter,1512691194
7286,CrisisFACTS-001-Twitter-45905-0,CrisisFACTS-001,#LilacFire Is the 76 closed both directions r...,Twitter,1512691195


In [9]:
# Create a filter expression
is_fb =  itemsAsDataFrame['source_type']=="Facebook"

# Apply our filter
itemsAsDataFrame[is_fb]

,doc_id,event,text,source_type,unix_timestamp
1188,CrisisFACTS-001-Facebook-0-0,CrisisFACTS-001,,Facebook,1512624253
1191,CrisisFACTS-001-Facebook-1-0,CrisisFACTS-001,,Facebook,1512624364
1192,CrisisFACTS-001-Facebook-1-1,CrisisFACTS-001,,Facebook,1512624364
1214,CrisisFACTS-001-Facebook-2-0,CrisisFACTS-001,,Facebook,1512625075
1215,CrisisFACTS-001-Facebook-2-1,CrisisFACTS-001,,Facebook,1512625075
...,...,...,...,...,...
7251,CrisisFACTS-001-Facebook-540-1,CrisisFACTS-001,,Facebook,1512691084
7252,CrisisFACTS-001-Facebook-541-0,CrisisFACTS-001,,Facebook,1512691086
7268,CrisisFACTS-001-Facebook-542-0,CrisisFACTS-001,,Facebook,1512691140
7269,CrisisFACTS-001-Facebook-542-1,CrisisFACTS-001,,Facebook,1512691140


In [10]:
# Create a filter expression
is_news =  itemsAsDataFrame['source_type']=="News"

# Apply our filter
itemsAsDataFrame[is_news]

,doc_id,event,text,source_type,unix_timestamp
0,CrisisFACTS-001-News-5-0,CrisisFACTS-001,Live updates: San Diego County fire is 92 perc...,News,1512604800
1,CrisisFACTS-001-News-5-1,CrisisFACTS-001,"The Lilac fire now 92 percent contained, Cal F...",News,1512604800
2,CrisisFACTS-001-News-5-2,CrisisFACTS-001,The county of San Diego has opened a Local Ass...,News,1512604800
3,CrisisFACTS-001-News-5-3,CrisisFACTS-001,The center is at the Vista branch library on 7...,News,1512604800
4,CrisisFACTS-001-News-5-4,CrisisFACTS-001,Homeowners also will be able to get informatio...,News,1512604800
...,...,...,...,...,...
6413,CrisisFACTS-001-News-45-40,CrisisFACTS-001,Authorities are trying to help large animals i...,News,1512688716
6414,CrisisFACTS-001-News-45-41,CrisisFACTS-001,Find evacuation information for large animals ...,News,1512688716
6415,CrisisFACTS-001-News-45-42,CrisisFACTS-001,Some social media reports indicate experienced...,News,1512688716
6416,CrisisFACTS-001-News-45-43,CrisisFACTS-001,"Horse Shows In The Sun's location in Thermal, ...",News,1512688716


You now have the data necessary for producing lists of facts for a given event-day pair.

<hr>

**Part 4. Queries for Disaster Manager/Responder Information Needs**

Clearly not all of the information in the input stream for each day will be useful for an emergency responder, or even be relevant. Hence it makes sense that we filter these streams down based on what the emergency responder cares about. Our task is focused on producing timeline summaries containing similar information to what might be entered into an after action report, similar to an ICS 209 form: 
* https://training.fema.gov/emiweb/is/icsresource/assets/ics%20forms/ics%20form%20209,%20incident%20status%20summary%20(v3).pdf

But how can we express this information need in a way that a computer can understand? 

To make it easier for participant systems to integrate content relevant to the event, we have manually constructed a set of queries that encapsulate this information need. These queries are in effect questions that an emergency responder might ask when writing their after action report.

These queries are included as part of each day of the CrisisFACTs dataset, and can access them as follows:

In [11]:
import pandas as pd

pd.DataFrame(dataset.queries_iter())

,query_id,text,trecis_category_mapping,event_id,event_title,event_dataset,event_description,event_trecis_id,event_type,event_url
0,CrisisFACTS-General-q001,Have airports closed,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
1,CrisisFACTS-General-q002,Have railways closed,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
2,CrisisFACTS-General-q003,Have water supplied been contaminated,Report-EmergingThreats,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
3,CrisisFACTS-General-q004,How many firefighters are active,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
4,CrisisFACTS-General-q005,How many people are affected,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
5,CrisisFACTS-General-q006,How many people are in shelters,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
6,CrisisFACTS-General-q007,How many people are missing,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
7,CrisisFACTS-General-q008,How many people are trapped,Request-SearchAndRescue,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
8,CrisisFACTS-General-q009,How many people have been injured,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire
9,CrisisFACTS-General-q010,How many people have been killed,Report-Factoid,CrisisFACTS-001,Lilac Wildfire 2017,2017_12_07_lilac_wildfire.2017,The Lilac Fire was a fire that burned in north...,TRECIS-CTIT-H-092,Wildfire,https://en.wikipedia.org/wiki/Lilac_Fire


<hr> 



## Searching for Relevant Content in the Newly Downloaded Data

At this point you know how to get the data streams that you are to summarize, and you know how what ideally should be included within your summary. This is the minimum that you need to tackle the CrisisFACTs task. However, one of the reasons that we integrated the CrisisFACTs datasets into ir_datasets is that it provides you with a plug-and-play means to perform text search of the content for a day via pyTerrier. This is useful as an initial step to find content that is relevant to the emergency responder information needs.

Before we get into creating our search engine, its worth providing a very broad overview of how a (text) search engine works. At its core, a search engine produces a data structure called an index from your document set. This index makes it really fast to identify documents containing a particular query term. To create our index, we need to provide the input documents, as well as specify what fields in the document contain text that we want to be searchable. 


**Part 1. Installing Packages**

Search engines are a core part of the online information space, and much work has gone into making this technology accessible and easy to develop. A major package in this space that is designed to facilitate experimentation with search and information-retrieval methods is `Terrier` and its related Python bindings. To use this library, we install the following packages:

*   pyTerrier (https://pyterrier.readthedocs.io/en/latest/): pyTerrier is a python wrapper around the Terrier IR Platform (a search engine in-a-box). We will use this to produce a searchable index for each day during a crisis event, so we can retrieve (hopefully) relevant content for different information needs. 

In [12]:
!pip install python-terrier # install pyTerrier

**Part 2. Creating an Index**

We can take one of the request strings for an <event,day> pair and ask pyTerrier to create an index for us:   

In [13]:
import pyterrier as pt

# Initalize pyTerrier if not started
if not pt.started():
  pt.init()

# Ask pyTerrier to download the dataset, the 'irds:' header tells pyTerrier to use ir_datasets as the data source
pyTerrierDataset = pt.get_dataset('irds:crisisfacts/001/2017-12-07')

# To create the index, we use an 'indexer', this interates over the documents in the collection and adds them to the index
# The paramters of this call are:
#  Index Storage Path: "None" (some index types write to disk, this would be the directory to write to)
#  Index Type: type=pt.index.IndexingType(3) (Type 3 is a Memory Index)
#  Meta Index Fields: meta=['docno', 'text'] (The index also can store raw fields so they can be attached to the search results, this specifies what fields to store)
#  Meta Index Lengths: meta_lengths=[40, 200] (pyTerrier allocates a fixed amount of storage space per field, how many characters should this be?)
indexer = pt.IterDictIndexer("None", type=pt.index.IndexingType(3), meta=['docno', 'text'], meta_lengths=[40, 200])

# Trigger the indexing process
index = indexer.index(pyTerrierDataset.get_corpus_iter())

PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

crisisfacts/001/2017-12-07 documents: 7288it [00:03, 2340.80it/s]


**Part 3. Handling Queries via the Retriever Object**

Now that we have an index, we can issue queries to it like you would do to a web search engine. Since this is our index, we have control over how we want scoring of the items to happen. Each item is scored using what is known as a weighting model. This is a function that produces a score based on the number of query terms the document contains, in combination with statistics of the documents in the dataset. Different weighting models are optimised for different types of documents. For instance, the classical BM25 model was designed for web pages.

In pyTerrier, we create a retriever object that will execute our queries. We pass the index to the retriever along with the weighting model we want to be used. We can also specify any raw fields we stored in the index for an item that we want to be attached to the search result, such as the original text:

In [14]:
retriever = pt.BatchRetrieve(index, wmodel="DFReeKLIM", metadata=["docno", "text"])

Now that we have a retriever, we can use it to issue queries:

In [15]:
pd.DataFrame(retriever.search("injuries"))

14:38:43.595 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


,qid,docid,docno,text,rank,score,query
0,1,1686,CrisisFACTS-001-News-26-18,A San Diego County Sheriff's Department deputy...,0,8.507087,injuries
1,1,6883,CrisisFACTS-001-Twitter-28187-0,Two civilians are being treated for burn injur...,1,8.493857,injuries
2,1,5246,CrisisFACTS-001-Twitter-879-0,Headon collision between Sheriff's vehicle and...,2,8.472945,injuries
3,1,6999,CrisisFACTS-001-Twitter-2-0,"UPDATE: #LilacFire now 2000 acres, 0% containm...",3,8.416696,injuries
4,1,3658,CrisisFACTS-001-Twitter-19765-0,Strong winds and low humidity fueling the Lila...,4,8.349830,injuries
5,1,236,CrisisFACTS-001-News-12-7,CAL FIRE/SAN DIEGO COUNTY FIRE (@CALFIRESANDI...,5,8.314384,injuries
6,1,2845,CrisisFACTS-001-News-38-35,"Besides the civilian injuries, one firefighter...",6,8.244784,injuries
7,1,1688,CrisisFACTS-001-News-26-20,It was not clear if it was a factor in the inj...,7,6.787178,injuries
